In [1]:
import ase
from ase.build import bulk
from ase.lattice.cubic import FaceCenteredCubic
from ase.calculators.lammpslib import LAMMPSlib
from ase.calculators import lj, morse
from ase import units
import chemiscope as chemi
import os
import numpy as np
import defects
import potentials

## Make a XYZ file ##

In [2]:
# Initial Data Parameters:

atom = 'Cu'
structure = 'fcc'
n_vacancies = 1 # Number of defects to be generated (removed atoms)
interstitial = 'Be'
fmax = 0.05 # maximum force tolerance for convergence

# Lattice parameters:
# r_lat0 = 3.60 * units.Angstrom # [Angstrom]
# E_co = 4.807e-14 * units.eV # [eV]
# Bulk_modulus = 1.33e+6 * units.bar # [bar]

# Results from DFT calculations:
JtoeV = 1.602176634e-19 # [eV/J]
epsilon = 6.567e-20 / JtoeV * units.eV # [eV]
sigma = 2.334 * units.Angstrom # [Meters]
cutoff_radius = 10.0 * units.Angstrom # [Angstrom]
D = 5.292e-20 / JtoeV * units.eV # epsilon for morse potential
alpha = 1.329 / units.Angstrom # A for morse potential
r0 = 2.885 * units.Angstrom # r_min for morse potential

# Interstitial parameters:
# between two interstitials
epsilon_int = 6.213e-20 * units.eV
sigma_int = 2.063 * units.Angstrom
D_int = 3.534e-20 / JtoeV * units.eV
alpha_int = 1.140 / units.Angstrom
r0_int = 2.898 * units.Angstrom

# between the main atom and an interstitial
epsilon_mix = (epsilon * epsilon_int) ** 0.5 
sigma_mix = (sigma + sigma_int) * 0.5
D_mix = 2 * D * D_int / (D + D_int)
alpha_mix = alpha * alpha_int * (alpha + alpha_int)/ (alpha ** 2 + alpha_int ** 2)
r0_mix = r0 * r0_int * (r0 + r0_int) / (r0 ** 2 + r0_int ** 2)

# dislocation parameters:
dislocation_plane = 'xz'
line_direction = 'y'
line_position = (0.5, 0.5, 0.5)

In [3]:
# Generate the ASE bulk data (unused):
# aseBulkData = bulk(
#     atom, 
#     structure, 
#     a=r_lat0,
#     cubic=True
# )

In [4]:
# Generate the ASE lattice data (used):
aseLatticeData = FaceCenteredCubic(
    directions=[[1,-1,0], [1,1,-2], [1,1,1]],
    size=(3,3,3),
    symbol='Cu',
    pbc=(1,1,1)
)

In [5]:
# Generate a directory for the atom:
if not os.path.exists(atom):
    os.makedirs(atom)
# inside the {atom} directory, generate a directory for the structure:
if not os.path.exists(f'{atom}/XYZ'):
    os.makedirs(f'{atom}/XYZ')
if not os.path.exists(f'{atom}/JSON'):
    os.makedirs(f'{atom}/JSON')
    

# Write the ASE data to a XYZ file:
XYZPath = f'{atom}/XYZ/{atom}_{structure}'
XYZBase = XYZPath + '.xyz'
aseLatticeData.write(XYZBase)

## Generate defects ##

In [6]:
defects.vacancies(
    atom,
    structure,
    aseLatticeData,
    n_vacancies
)
defects.dislocations(
    atom,
    structure,
    aseLatticeData,
    dislocation_plane=dislocation_plane,
    line_direction=line_direction,
    line_position=line_position
)
defects.interstitials(
    atom,
    structure,
    aseLatticeData,
    interstitial
)

XYZVacancy = '_vacancy.xyz'
XYZDislocation = '_dislocation.xyz'
XYZInterstitial = '_interstitial.xyz'

Removed atom 62 from Cu_fcc_vacancy.xyz


## Calculate Potentials ##

In [7]:
XYZDefects = {
    'Vacancy': XYZVacancy,
    'Dislocation': XYZDislocation, 
    'Interstitial': XYZInterstitial
}

# the frames dictionnary starts with the base structure
frames = {'Base': ase.io.read(XYZBase, ':')}

# two frames are further generated for each defect
for label in XYZDefects:

    # one frame will be relaxed (denoted by R)
    frames['R' + label] = ase.io.read(XYZPath + XYZDefects[label], ':')

    # while one frame won't
    frames['N' + label] = ase.io.read(XYZPath + XYZDefects[label], ':')

properties = {}

for label in frames:

    settings = {
        'frame': frames[label][0],
        'cutoff_radius': cutoff_radius,
        'parameters': [[epsilon, sigma]],
        'atoms': [atom],
        'relax': True if label[0] == 'R' else False,
        'fmax': fmax
    }

    if label[1:] == 'Interstitial':
        settings['parameters'].extend([[epsilon_int, sigma_int], [epsilon_mix, sigma_mix]])
        settings['atoms'].extend([interstitial])

    Charges = frames[label][0].get_atomic_numbers()

    LennardJones = potentials.LAMMPS(
        potential="lj/cut",
        settings=settings
    )

    settings['parameters'] = [[D, alpha, r0]]
    if label[1:] == 'Interstitial':
        settings['parameters'].extend([[D_int, alpha_int, r0_int], [D_mix, alpha_mix, r0_mix]])

    Morse = potentials.LAMMPS(
        potential="morse",
        settings=settings
    )

    # LJTemp = potentials.LJ(
    #     frame=frames[label],
    #     sigma=sigma,
    #     epsilon=epsilon
    # )

    properties[label] = [{**LennardJones, **Morse}]

# print(properties)

Atoms(symbols='Cu162', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094])
Atoms(symbols='Cu162', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094], calculator=LAMMPSlib(...))
Atoms(symbols='Cu161', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094])
       Step     Time          Energy          fmax
LBFGS:    0 20:20:56     -557.213938        0.397019
LBFGS:    1 20:20:56     -557.236032        0.226834
LBFGS:    2 20:20:56     -557.248951        0.056208


LBFGS:    3 20:20:56     -557.250197        0.045488
Atoms(symbols='Cu161', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094], calculator=LAMMPSlib(...))
       Step     Time          Energy          fmax
LBFGS:    0 20:20:56     -560.519210        0.587926
LBFGS:    1 20:20:56     -560.574871        0.478133
LBFGS:    2 20:20:56     -560.681220        0.126800
LBFGS:    3 20:20:56     -560.687043        0.110140
LBFGS:    4 20:20:56     -560.704703        0.066491
LBFGS:    5 20:20:56     -560.706348        0.058382
LBFGS:    6 20:20:56     -560.710693        0.026825
Atoms(symbols='Cu161', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094])
Atoms(symbols='Cu161', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094], calculator=LAMMPSlib(...))
Atoms(symbols='Cu162', pbc=True, cell=[7.65796644025031, 13.263986957170907, 18.75811024597094])
       Step     Time          Energy          fmax
LBFGS:    0 20:20:57 15817820511

In [9]:
JSONPath = f"{atom}/JSON/{atom}_{structure}"
path = JSONPath + '.json.gz'
DatasetInformation = {
    'name': atom + ' ' + structure,
    'description': 'Semester Project 1, atom ' + atom + ' in ' + structure + ' structure',
    'authors': [
        'Maxime Saillen',
    ],
    'references': [
        'https://doi.org/10.1016/j.commatsci.2022.111206',
    ],
}

for label in frames:

    if label[0] == 'R':
        DatasetInformation['name'] += ' relaxed'
        DatasetInformation['description'] += ' relaxed'
        JSONPath_new = JSONPath + 'R'
    else:
        JSONPath_new = JSONPath

    if label[1:] == 'Vacancy':
        DatasetInformation['name'] += ' with ' + str(n_vacancies) + ' vacancies'
        DatasetInformation['description'] += ' with ' + str(n_vacancies) + ' vacancies'
        path = JSONPath_new + '_vacancy.json.gz'

    elif label[1:] == 'Dislocation':
        DatasetInformation['name'] += ' with a dislocation along the ' + str(line_direction) + ' axis'
        DatasetInformation['description'] += ' with a dislocation along the ' + str(line_direction) + ' axis'
        path = JSONPath_new + '_dislocation.json.gz'

    elif label[1:] == 'Interstitial':
        DatasetInformation['name'] += ' with a ' + str(interstitial) + ' interstitial'
        DatasetInformation['description'] += ' with a ' + str(interstitial) + ' interstitial'
        path = JSONPath_new + '_interstitial.json.gz'

    chemi.write_input(
        path=path,
        frames=frames[label][0], 
        meta=DatasetInformation, 
        properties=properties[label][0], 
        environments=chemi.all_atomic_environments(frames[label][0]), 
        settings=None, 
        shapes=None, 
        parameters=None
    )